In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name="llama-3.1-70b-versatile"
)

In [5]:
response=llm.invoke("What is Ichigo's mother's history")
print(response.content)

Ichigo Kurosaki's mother, Masaki Kurosaki, was a Quincy. She was the wife of Isshin Kurosaki, a Shinigami (Soul Reaper), and the mother of Ichigo and his sisters, Yuzu and Karin.

Masaki was a member of the Quincy clan, a group of spiritually aware humans who possessed the ability to absorb and manipulate spiritual energy. However, she fell in love with Isshin, who was a Shinigami at the time, and the two got married.

Tragically, Masaki was killed by a Hollow, a malevolent spirit, when Ichigo was just a child. This event had a profound impact on Ichigo's life and played a significant role in shaping his personality and motivations.

It was later revealed that Masaki's death was not just a random attack by a Hollow, but was actually orchestrated by the Arrancar, Grand Fisher, who was seeking to avenge the Quincy clan's persecution by the Shinigami.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39039")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Data Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Data Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. Team
NIKE, I

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template( 
     """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    #### INSTRUCTION:
    The scraped text is from the career's page of a website.
     Your job is to extract the job postings and return them in JSON format containing the following keys : "role","experience","skills" and "Description .Only return the valid json
     ### VALID JSON (NO PREAMBLE) and also no ```json ticks         
   
    #           
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke({"page_data":page_data})
print(res.content)

{
  "role": "Senior Data Engineer, ITC",
  "experience": "5+ years of hands on experience in data engineering field",
  "skills": [
    "Bachelors degree in Computer Science engineering",
    "In depth big data tech stack knowledge",
    "Expertise in pyspark, sql and data modelling",
    "Expertise in databricks, snowflake, EMR, airflow",
    "Pipeline design and architectural skillset",
    "Knowledge on performance tuning of big data pipelines",
    "Excellent written and verbal communication skills",
    "Mentoring and guiding skills"
  ],
  "Description": "A senior data engineer will work in the Data and Artificial Intelligence organisation of Nike and will focus on building highly complex and performant data pipelines that'll drive Nike's data driven strategies for the future of sports."
}


In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parsor = JsonOutputParser()
json_res = json_parsor.parse(res.content)
json_res

{'role': 'Senior Data Engineer, ITC',
 'experience': '5+ years of hands on experience in data engineering field',
 'skills': ['Bachelors degree in Computer Science engineering',
  'In depth big data tech stack knowledge',
  'Expertise in pyspark, sql and data modelling',
  'Expertise in databricks, snowflake, EMR, airflow',
  'Pipeline design and architectural skillset',
  'Knowledge on performance tuning of big data pipelines',
  'Excellent written and verbal communication skills',
  'Mentoring and guiding skills'],
 'Description': "A senior data engineer will work in the Data and Artificial Intelligence organisation of Nike and will focus on building highly complex and performant data pipelines that'll drive Nike's data driven strategies for the future of sports."}

In [18]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")

In [20]:
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [22]:
import chromadb
import uuid

client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links":row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [35]:
links=collection.query(query_texts=["Experience in Python","Experience in React Native"],n_results = 2).get("metadatas",[])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [41]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB description
    {job_description}
    ### INSTRUCTION:
        You are Mahesh, a business development executive at CronAi. CronAi is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency.

        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CronAi technologies 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase CronAi's portfolio: {link_list}
        Remember you are Mahesh, BDE at CronAi. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE)
    """
)

job = json_res
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job),"link_list":links})


In [42]:

print(res.content)

Subject: Expert Data Engineering Solutions for Nike's Data-Driven Strategies

Dear Hiring Manager,

I came across the Senior Data Engineer, ITC role at Nike and was impressed by the organization's commitment to leveraging data-driven strategies for the future of sports. As a Business Development Executive at CronAi, I'd like to introduce our company as a trusted partner in delivering cutting-edge data engineering solutions.

At CronAi, we specialize in designing and implementing scalable, high-performance data pipelines that drive business growth. Our team of experts has extensive experience in big data technologies, including PySpark, SQL, and data modeling. We've successfully developed and deployed data pipelines on various platforms, such as Databricks, Snowflake, EMR, and Airflow.

Our expertise in pipeline design and architectural skillset, combined with our knowledge of performance tuning, ensures that our solutions meet the highest standards of efficiency and reliability. We're 